In [35]:
import os
import getpass
from openai import OpenAI

# Digite sua chave
openai_key = getpass.getpass("Digite sua chave OpenAI: ")

# Define a variável de ambiente
os.environ["OPENAI_API_KEY"] = openai_key

In [36]:
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

with open('../data/processed/crm_conversas_anonimizadas.txt', 'r', encoding='utf-8') as f:
    text = f.read()

text = text[:100_000]  # só os primeiros 100 mil caracteres (~15 mil palavras)

docs = [Document(page_content=text)]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,        # menor chunk para menos tokens
    chunk_overlap=200,      # overlap menor para menos repetição
    add_start_index=True
)

all_splits = text_splitter.split_documents(docs)

print(f"Total de chunks criados: {len(all_splits)}")


Total de chunks criados: 82


In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

batch_size = 10  # bem leve
vectorstores = []

for i in range(0, len(all_splits), batch_size):
    batch = all_splits[i:i + batch_size]
    store = FAISS.from_documents(batch, embeddings)
    vectorstores.append(store)

# Juntar todos os vetores se necessário
vectorstore = vectorstores[0]
for store in vectorstores[1:]:
    vectorstore.merge_from(store)


In [39]:
vectorstore.save_local("meus_embeddings/")

In [40]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("Qual o maior motivo de insatisfação?")

len(retrieved_docs)

6

In [42]:
print(retrieved_docs[1].page_content)

<br />08:42
<br />
<br />Resposta
<br />
<br />E registra minha reclamação dessa atualização da biometria
<br />
<br />Você pode abrir um CRM informando a sua insatisfação com a biometria.
<br />
<br />Você vai no Portal, clica em ?Outros Serviços? ou em qualquer opção que abra o S4E. Aí vai aparecer uma barra de ferramentas, na qual você escolherá a opção ?CRM?(Ícone de balão colorido). Depois você clica no sinal de + (verde) para abrir o chamado. Lá você coloca a solicitação e descreve a situação.
<br />
<br />
<br />Resposta
<br />
<br />Não recebi o kit pq não entrou !!
<br />
<br />Como lhe informei anteriormente. O procedimento só pode realizado após a baixa do procedimento em sistema. Caso você esteja com dificuldade na validação da biometria você precisa contatar a central do dentista de imediato para que possamos lhe auxiliar
<br />
<br />Posso te ajudar com mais alguma informação?
<br />
<br />08:43
<br />Espero ter ajudado!
<br />
<br />Agora vou te transferir para a pesquis

In [43]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [44]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [45]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [46]:
for chunk in rag_chain.stream("Qual a insatisfação dos clientes?"):
    print(chunk, end="", flush=True)

Os clientes estão insatisfeitos principalmente com o atendimento, dificuldades na biometria e falta de informações sobre procedimentos e pagamentos. Além disso, houve reclamações sobre o agendamento de consultas e o não cumprimento de horários.